# Aufgabe 4

In [1]:
import numpy as np

In [2]:
from microtorch.tensor import Tensor
from microtorch import nn

In [3]:
x = Tensor([[3.0, -1.0]])

y = Tensor.ones((1, 1))

In [4]:
w1 = Tensor([
    [0.5, 1.0, 2.0],
    [1.0, 3.0, 6.0]
])

w2 = Tensor([
    [ 2.0],
    [-1.0],
    [ 0.8]
])

In [5]:
w1

Tensor([[0.5, 1. , 2. ],
        [1. , 3. , 6. ]])

In [6]:
f1   = x@w1
f2   = f1@w2
sig  = f2.sigmoid()
loss = sig.l2(y)

sig, loss

(Tensor([[0.73105858]]), Tensor(0.07232949))

In [7]:
η = 1.0

x.zero_grad()
w1.zero_grad(); f1.zero_grad()
w2.zero_grad(); f2.zero_grad()
sig.zero_grad()
loss.zero_grad()
loss.backward()

w1.data -= η * w1.grad
w2.data -= η * w2.grad

In [8]:
w1

Tensor([[1.13452511, 0.68273744, 2.25381005],
        [0.78849163, 3.10575419, 5.91539665]])

In [9]:
w2

Tensor([[ 2.05287709],
        [-1.        ],
        [ 0.8       ]])

# XOR

In [10]:
x = Tensor([
    [0.0, 0.0],
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0]
])

y = np.array([
    [0.0],
    [1.0],
    [1.0],
    [0.0],
])

In [11]:
model = nn.Sequential(
    nn.Linear(2, 16),
    nn.LeakyReLU(0.01),
    nn.Linear(16, 16),
    nn.LeakyReLU(0.01),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

In [12]:
def optimize(params, lr=0.001):
    for p in params:
        p.data -= lr * p.grad
        
def zero_grad(*params):
    for p in params:
        p.zero_grad()

In [13]:
for i in range(1000):
    loss = Tensor.l2(model(x), y)
    
    zero_grad(*model.params(), x)
    
    loss.backward()
    
    optimize(model.params(), lr=0.1)

In [14]:
pred = model(x)

In [32]:
pred

Value([[0.00942888],
       [0.99233796],
       [0.99267392],
       [0.007169  ]])